In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import requests
import pandas as pd
import numpy as np
import time
import xgboost as xgb
import torch
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error

# 🚀 Load AI Model for Reasoning (Zephyr-3B)
device = "cuda" if torch.cuda.is_available() else "cpu"
model_name = "stabilityai/stablelm-zephyr-3b"

# ✅ Load tokenizer & model
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    device_map="auto"
)
model.config.pad_token_id = tokenizer.eos_token_id

# 🌍 NASA Exoplanet Archive API
NASA_ARCHIVE_URL = (
    "https://exoplanetarchive.ipac.caltech.edu/TAP/sync?"
    "query=select+pl_name,pl_rade,pl_bmasse,pl_orbper,pl_eqt,st_teff,st_mass,st_rad,st_met+from+pscomppars&format=json"
)

# 🔄 Fetch Data
def fetch_data(url, retries=3, delay=5):
    for attempt in range(retries):
        try:
            print(f"[Attempt {attempt + 1}] Fetching data from NASA API...")
            response = requests.get(url, timeout=20)
            response.raise_for_status()
            data = response.json()
            if isinstance(data, list) and data:
                return data
        except requests.exceptions.RequestException as e:
            print(f"[Error] Request failed: {e}")
        time.sleep(delay)
    return []

# 💼 Fetch NASA Data
nasa_data = fetch_data(NASA_ARCHIVE_URL)
df = pd.DataFrame(nasa_data)
if df.empty:
    raise ValueError("[Fatal] NASA data fetch failed. Cannot proceed.")

# 📊 Data Preprocessing
numeric_columns = ['pl_rade', 'pl_bmasse', 'pl_orbper', 'pl_eqt', 'st_teff', 'st_mass', 'st_rad', 'st_met']
df[numeric_columns] = df[numeric_columns].apply(pd.to_numeric, errors='coerce')
df.dropna(subset=numeric_columns, inplace=True)

# ⚙️ Additional Factors Calculations
def atmospheric_composition_score(row):
    return max(0, min(20, (row['st_met'] + 0.5) * 15))

def liquid_water_probability(row):
    return max(0, min(30, (300 - abs(row['pl_eqt'] - 300)) / 3)) if 200 <= row['pl_eqt'] <= 400 else 0

def magnetosphere_strength(row):
    return max(0, min(25, (row['st_mass'] / row['pl_bmasse']) * 10))

def stellar_radiation_factor(row):
    return max(0, min(25, (6000 - row['st_teff']) / 100)) if row['st_teff'] < 6000 else 0
    

# ⚙️ Habitability Scoring Function
def rule_based_score(row):
    if row['pl_rade'] > 3:
        return 0
    mass_component = min(20, row['pl_bmasse'] * 5)  
    temp_component = max(0, (300 - abs(row['pl_eqt'] - 300)) / 3) if 200 <= row['pl_eqt'] <= 400 else 0
    return min(100, mass_component + temp_component)

df['habitability_score'] = df.apply(rule_based_score, axis=1)

# ⚙️ Terraformability Scoring Function
def terraformability_score(row):
    if row['pl_eqt'] < 200 or row['pl_eqt'] > 400:
        return 0
    atmosphere_factor = max(0, min(30, (row['st_met'] + 0.5) * 20))
    gravity_factor = max(0, min(30, 9.8 / (row['pl_bmasse'] ** 0.5)))
    temp_factor = max(0, (300 - abs(row['pl_eqt'] - 300)) / 3)
    return min(100, atmosphere_factor + gravity_factor + temp_factor)

df['terraformability_score'] = df.apply(terraformability_score, axis=1)

# 📈 Train XGBoost Model
X, y = df[numeric_columns], df['habitability_score']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

xgb_model = xgb.XGBRegressor(n_estimators=300, learning_rate=0.05, max_depth=7)
xgb_model.fit(X_train, y_train)

# ✅ Model Accuracy
y_pred = xgb_model.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)
print(f"\n✅ Model trained successfully! MAE: {mae:.4f}\n")

# 🧠 AI Reasoning with Terraformability (Fixed)
def generate_reasoning(planet_data, habitability, terraformability):
    prompt = f"""
    You are an AI astrophysicist analyzing exoplanet habitability and terraformability.

    Analyze the habitability and terraformability of {planet_data['pl_name']}:

    - Mass: {planet_data['pl_bmasse']} Earth masses
    - Radius: {planet_data['pl_rade']} Earth radii
    - Temperature: {planet_data['pl_eqt']} K
    - Orbital Period: {planet_data['pl_orbper']} days
    - Host Star Temperature: {planet_data['st_teff']} K
    - Stellar Mass: {planet_data['st_mass']} Solar masses
    - Metallicity: {planet_data['st_met']} [Fe/H]

    Habitability Score: {habitability}%
    Terraformability Score: {terraformability}%

    Explain these scores using planetary physics, atmospheric retention, surface gravity, and habitability criteria.
    Do NOT generate Python code. Only provide a scientific explanation.
    """

    inputs = tokenizer(prompt, return_tensors='pt', padding=True, truncation=True).to(device)
    with torch.no_grad():
        tokens = model.generate(
            inputs.input_ids,
            attention_mask=inputs.attention_mask,
            max_new_tokens=512, 
            temperature=0.7, 
            do_sample=True,
            top_p=0.9, 
            repetition_penalty=1.1
        )
    return tokenizer.decode(tokens[0], skip_special_tokens=True)

# 🎯 Main Program
def main():
    while True:
        user_input = input("\n🔍 Enter exoplanet name (or 'exit' to quit): ").strip().lower()
        if user_input == 'exit':
            print("\n🚀 Exiting program. Goodbye!\n")
            break

        planet_data = df[df['pl_name'].str.lower() == user_input]
        if planet_data.empty:
            print("\n❌ Planet not found. Try another name.\n")
            continue

        planet_data = planet_data.iloc[0]
        ml_score = xgb_model.predict(pd.DataFrame([planet_data[numeric_columns]]))[0]
        terraformability = terraformability_score(planet_data)

        mass_component = min(20, planet_data['pl_bmasse'] * 5)
        temp_component = liquid_water_probability(planet_data)
        atmosphere_component = atmospheric_composition_score(planet_data)
        radiation_component = stellar_radiation_factor(planet_data)
        magnetosphere_component = magnetosphere_strength(planet_data)

        print(f"\n🌍 **{planet_data['pl_name']} Analysis:**")
        print(f"📊 ML Habitability Prediction: {ml_score:.2f}%")
        print(f"📏 Rule-Based Habitability Score: {planet_data['habitability_score']}%")
        print(f"🌏 Terraformability Score: {terraformability}%")

        print("\n🔬 **Factor Contributions to Habitability Score:**")

        total_score = mass_component + temp_component + atmosphere_component + radiation_component + magnetosphere_component
        print(f"🪐 Mass Component: {mass_component / total_score * 100:.2f}%")
        print(f"🌡️ Temperature Component: {temp_component / total_score * 100:.2f}%")
        print(f"☁️ Atmospheric Composition: {atmosphere_component / total_score * 100:.2f}%")
        print(f"☀️ Stellar Radiation Factor: {radiation_component / total_score * 100:.2f}%")
        print(f"🧲 Magnetosphere Strength: {magnetosphere_component / total_score * 100:.2f}%")
        
        print("\n🧠 AI Reasoning:") 
        print(generate_reasoning(planet_data, ml_score, terraformability))
        print("=================================")

main()


[Attempt 1] Fetching data from NASA API...

✅ Model trained successfully! MAE: 0.3858




🔍 Enter exoplanet name (or 'exit' to quit):  Kepler-442 b


Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.



🌍 **Kepler-442 b Analysis:**
📊 ML Habitability Prediction: 92.17%
📏 Rule-Based Habitability Score: 92.13333333333333%
🌏 Terraformability Score: 89.31258997139369%

🔬 **Factor Contributions to Habitability Score:**
🪐 Mass Component: 18.94%
🌡️ Temperature Component: 48.14%
☁️ Atmospheric Composition: 3.13%
☀️ Stellar Radiation Factor: 25.64%
🧲 Magnetosphere Strength: 4.15%

🧠 AI Reasoning:

    You are an AI astrophysicist analyzing exoplanet habitability and terraformability.

    Analyze the habitability and terraformability of Kepler-442 b:

    - Mass: 2.36 Earth masses
    - Radius: 1.34 Earth radii
    - Temperature: 241.0 K
    - Orbital Period: 112.3053 days
    - Host Star Temperature: 4402.0 K
    - Stellar Mass: 0.61 Solar masses
    - Metallicity: -0.37 [Fe/H]

    Habitability Score: 92.16641998291016%
    Terraformability Score: 89.31258997139369%

    Explain these scores using planetary physics, atmospheric retention, surface gravity, and habitability criteria.
    Do NO